In [1]:
push!(LOAD_PATH,joinpath(@__DIR__, "../src/"))
using LinearAlgebra
using StableApproxEPW

## Target of the approximation problem

We consider the Helmholtz solution in the unit disk, with wavenumber

In [2]:
k = 10;

We need to define the maximum Fourier mode number in the approximation target

In [3]:
P = 15;

Next we construct the vector of coefficients in the basis `b_p`
for `p` in `[-P,P]`.

In [4]:
U = zeros(ComplexF64, 2P+1);
U[P+1]     = 0.5;  # This is the constant mode (`p=0`)
U[P+1 + P] = 1im;  # This is mode `p = P`

The target of the approximation problem can then be constructed as

In [5]:
u = solution_surrogate(U; k=k);

``u`` can be evaluated at any `(r,θ)` point.
Alternatively, the target ``u`` could have been a single mode.
For instance, to get the circular wave with mode number `p=15`, simply set
``u = bp(15; k=k)``.
Of course, any other function (defined in polar coordinates) can be defined
by the user as target of the approximation problem.

## Reconstruction method

In [6]:
N = 40;

Sampling nodes

In [7]:
S = number_of_boundary_sampling_nodes(N; η=2, P=P);
X = boundary_sampling_nodes(S);

Right-hand-side

In [8]:
b = samples_from_nodes(u, X);

## Approximation with **propagative** plane waves

### Approximation set

In [9]:
Φppw = approximation_set(N; k=k);

Matrix and its factorization

In [10]:
Appw = samples_from_nodes(Φppw, X);

### Solution

In [11]:
iAppw = RegularizedSVDPseudoInverse(Appw; ϵ=1e-12);

The coefficients are computed using the

In [12]:
ξppw = solve_via_regularizedSVD(iAppw, b);
ũppw = (r,θ) -> sum([ξi * ϕi(r,θ) for (ξi, ϕi) in zip(ξppw, Φppw)]);

### Errors and stability estimate

Relative residual

In [13]:
resppw = norm(Appw * ξppw - b) / norm(b)

1.564762588352948e-6

Stability measure

In [14]:
nrmppw = norm(ξppw) / norm(U)

179.99923086555276

Absolute error function

In [15]:
eppw = (r,θ) -> ũppw(r,θ) - u(r,θ);

## Approximation with **evanescent** plane waves

### Approximation set

In [16]:
Φepw = approximation_set(N, P, sobol_sampling; k=k);

Matrix and its factorization

In [17]:
Aepw = samples_from_nodes(Φepw, X);

### Solution

In [18]:
iAepw = RegularizedSVDPseudoInverse(Aepw; ϵ=1e-12);

The coefficients are computed using the

In [19]:
ξepw = solve_via_regularizedSVD(iAepw, b);
ũepw = (r,θ) -> sum([ξi * ϕi(r,θ) for (ξi, ϕi) in zip(ξepw, Φepw)]);

### Errors and stability estimate

Relative residual

In [20]:
resepw = norm(Aepw * ξepw - b) / norm(b)

0.10591731601117463

Stability measure

In [21]:
nrmepw = norm(ξepw) / norm(U)

58.36892810200745

Absolute error function

In [22]:
eepw = (r,θ) -> ũepw(r,θ) - u(r,θ);

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*